<a href="https://colab.research.google.com/github/anash18/4105-HW-Nash/blob/main/ECGR_4105_HW5/ECGR_4105_HW5_Problem_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim

In [73]:
#Calling the dataset
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/4105 Datasets/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
housing.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [74]:
#Applying a binary map
#mapping the variables to a binary output
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def mapping(x):
    return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

housing[varlist] = housing[varlist].apply(mapping)

housing.head()
#housing.dtypes

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1


In [75]:
#Splitting the Dataset
x3_train, x3_val = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 0)
x3_train.head()

#Splitting Variables
y3_train = x3_train.pop("price")
y3_val = x3_val.pop("price")

#print(x3_train)
#print(y3_train)
#x3_train.head()
#print(y3_val)

In [76]:
#Standardizing the Dataset

standard = StandardScaler()

vars3 = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating',
         'airconditioning', 'prefarea', 'furnishingstatus']

x3_train_stan = x3_train
x3_train_stan[vars3] = standard.fit_transform(x3_train_stan[vars3])

x3_val_stan = x3_val
x3_val_stan[vars3] = standard.fit_transform(x3_val_stan[vars3])
#x3_val_stan.head()

In [77]:
#Stacking them there inputs
x3_train_stan_numpy = np.c_[np.ones((len(y3_train), 1)), x3_train_stan[vars3]]
x3_val_stan_numpy = np.c_[np.ones((len(y3_val), 1)), x3_val_stan[vars3]]

#I am become tensor destroyer of deep learning
x3_train_ten = torch.tensor(x3_train_stan_numpy)
x3_val_ten = torch.tensor(x3_val_stan_numpy)
y3_train_ten = torch.tensor(y3_train.values)
y3_val_ten = torch.tensor(y3_val.values)


y3_train_mean = y3_train_ten.float().mean()
y3_train_std = y3_train.std()
y3_train_stan_ten = (y3_train_ten-y3_train_mean) / y3_train_std

y3_val_mean = y3_val_ten.float().mean()
y3_val_std = y3_val.std()
y3_val_stan_ten = (y3_val_ten-y3_val_mean) / y3_val_std

In [78]:
#Getting lost and starting my modeling career

#defining the model
def model(t_u, w12, w11, w10, w9, w8, w7, w6, w5, w4, w3, w2, w1, b):
  return w12*t_u**12 + w11*t_u**11 + w10*t_u**10 + w9*t_u**9 + w8*t_u**8 + w7*t_u**7 + w6*t_u**6 + w5*t_u**5 + w4*t_u**4 + w3*t_u**3 + w2 * t_u ** 2 + w1 * t_u + b

#defining the loss function
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

#Setting up the initial parameters
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)

In [79]:
#Time for an anime training arc

#defining a training loop
def training_loop(n_epochs, optimizer, params, t_u_train, t_c_train, t_u_val, t_c_val):
  for epoch in range (1, n_epochs +1):

    if params.grad is not None:  # <1>
      params.grad.zero_()

    t_p_train = model(t_u_train, *params)
    loss_train = loss_fn(t_p_train.transpose(0,1), t_c_train)

    t_p_val = model(t_u_val, *params)
    loss_val = loss_fn(t_p_val.transpose(0,1), t_c_val)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}\n")
  return params


#SGD Number 1 learning rate of 0.000000000000001
learning_rate_SGD1 = 0.000000000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD1 = optim.SGD([params], lr=learning_rate_SGD1)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD1,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_ten,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 131467027132112.6875, Validation loss: 35936793892416.8516

Epoch 3, Training loss: 67211620587508.2656, Validation loss: 28966503476866.1055

Epoch 500, Training loss: 26316996983323.4141, Validation loss: 25063440564244.7383

Epoch 1000, Training loss: 26316956615056.3164, Validation loss: 25063284860056.3477

Epoch 1500, Training loss: 26316916514058.9570, Validation loss: 25063130131141.7656

Epoch 2000, Training loss: 26316876500879.4570, Validation loss: 25062975467602.3086

Epoch 2500, Training loss: 26316836567309.3242, Validation loss: 25062820861225.9062

Epoch 3000, Training loss: 26316796809349.3477, Validation loss: 25062666846880.2461

Epoch 3500, Training loss: 26316757288790.0195, Validation loss: 25062513451609.3086

Epoch 4000, Training loss: 26316717961016.8789, Validation loss: 25062360467784.6680

Epoch 4500, Training loss: 26316678711317.9961, Validation los

tensor([-1.8183e-01,  7.9599e-01,  9.8157e-01,  9.9976e-01,  1.0029e+00,
         1.0008e+00,  1.0005e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00,  4.7538e-05], requires_grad=True)

In [80]:
#SGD Number 2 learning rate of 0.0000000000000001
learning_rate_SGD2 = 0.0000000000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD2 = optim.SGD([params], lr=learning_rate_SGD2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD2,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_stan_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_stan_ten,
)

Epoch 1, Training loss: 282610389830254.3125, Validation loss: 33358261732576.9219

Epoch 2, Training loss: 261737851747342.9062, Validation loss: 30909229952786.1641

Epoch 3, Training loss: 242406895492662.9688, Validation loss: 28640529653542.3867

Epoch 500, Training loss: 3484939676.9695, Validation loss: 2622778678.7937

Epoch 1000, Training loss: 3483412118.6856, Validation loss: 2621592846.2406

Epoch 1500, Training loss: 3481884623.8765, Validation loss: 2620488216.2241

Epoch 2000, Training loss: 3480357464.6309, Validation loss: 2619383819.5188

Epoch 2500, Training loss: 3478830640.9487, Validation loss: 2618279656.1248

Epoch 3000, Training loss: 3477304169.4247, Validation loss: 2617170968.4624

Epoch 3500, Training loss: 3475778016.8430, Validation loss: 2616067272.7374

Epoch 4000, Training loss: 3474252199.8248, Validation loss: 2614963810.3236

Epoch 4500, Training loss: 3472726734.9654, Validation loss: 2613855826.7898

Epoch 5000, Training loss: 3471201589.0476, Val

tensor([-2.0004e-01,  7.5684e-01,  9.5052e-01,  9.9001e-01,  9.9805e-01,
         9.9961e-01,  9.9992e-01,  9.9998e-01,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00, -7.1321e-09], requires_grad=True)

In [81]:
#SGD Number 3 learning rate of 0.00000000000000001
learning_rate_SGD3 = 0.00000000000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD3 = optim.SGD([params], lr=learning_rate_SGD3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD3,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_stan_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_stan_ten,
)

Epoch 1, Training loss: 282610389830254.3125, Validation loss: 33358261732576.9219

Epoch 2, Training loss: 280487120080746.2188, Validation loss: 33109158747414.0078

Epoch 3, Training loss: 278379792600305.5938, Validation loss: 32861920452775.5820

Epoch 500, Training loss: 6562343773986.7930, Validation loss: 828522906624.5294

Epoch 1000, Training loss: 154563178131.3636, Validation loss: 29509339832.5713

Epoch 1500, Training loss: 6965910785.1996, Validation loss: 4435958753.9926

Epoch 2000, Training loss: 3566052852.0215, Validation loss: 2846251104.3841

Epoch 2500, Training loss: 3487573557.5863, Validation loss: 2655905616.1969

Epoch 3000, Training loss: 3485640526.2753, Validation loss: 2628295886.4522

Epoch 3500, Training loss: 3485454356.2959, Validation loss: 2623944442.6665

Epoch 4000, Training loss: 3485298823.9780, Validation loss: 2623294399.2399

Epoch 4500, Training loss: 3485142552.9283, Validation loss: 2623182729.4801

Epoch 5000, Training loss: 3484986285.3

tensor([-2.0051e-01,  7.5897e-01,  9.5164e-01,  9.9030e-01,  9.9806e-01,
         9.9961e-01,  9.9993e-01,  9.9999e-01,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00, -5.4258e-09], requires_grad=True)

In [82]:
#SGD Number 4 learning rate of 00.000000000000000001
learning_rate_SGD4 = 0.00000000000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD4 = optim.SGD([params], lr=learning_rate_SGD4)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD4,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_stan_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_stan_ten,
)

Epoch 1, Training loss: 282610389830254.3125, Validation loss: 33358261732576.9219

Epoch 2, Training loss: 280487120080746.2188, Validation loss: 33109158747414.0078

Epoch 3, Training loss: 278379792600305.5938, Validation loss: 32861920452775.5820

Epoch 500, Training loss: 6562343773986.7930, Validation loss: 828522906624.5294

Epoch 1000, Training loss: 154563178131.3636, Validation loss: 29509339832.5713

Epoch 1500, Training loss: 6965910785.1996, Validation loss: 4435958753.9926

Epoch 2000, Training loss: 3566052852.0215, Validation loss: 2846251104.3841

Epoch 2500, Training loss: 3487573557.5863, Validation loss: 2655905616.1969

Epoch 3000, Training loss: 3485640526.2753, Validation loss: 2628295886.4522

Epoch 3500, Training loss: 3485454356.2959, Validation loss: 2623944442.6665

Epoch 4000, Training loss: 3485298823.9780, Validation loss: 2623294399.2399

Epoch 4500, Training loss: 3485142552.9283, Validation loss: 2623182729.4801

Epoch 5000, Training loss: 3484986285.3

tensor([-2.0051e-01,  7.5897e-01,  9.5164e-01,  9.9030e-01,  9.9806e-01,
         9.9961e-01,  9.9993e-01,  9.9999e-01,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00, -5.4258e-09], requires_grad=True)

In [68]:
#Adam Number 1 learning rate of 0.1
learning_rate_Adam1 = 0.1
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam1 = optim.Adam([params], lr=learning_rate_Adam1)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam1,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_ten,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 244227149707464.5938, Validation loss: 48444982803921.2422

Epoch 3, Training loss: 197610155763582.6562, Validation loss: 43213275408720.7422

Epoch 500, Training loss: 26329257847778.0625, Validation loss: 25104353016655.0273

Epoch 1000, Training loss: 26320920173619.9219, Validation loss: 25087244782815.9961

Epoch 1500, Training loss: 26310415886434.2461, Validation loss: 25064677439609.2070

Epoch 2000, Training loss: 26298627027275.3945, Validation loss: 25038110880296.0664

Epoch 2500, Training loss: 26286369032988.8906, Validation loss: 25009113949460.4531

Epoch 3000, Training loss: 26274287907440.1406, Validation loss: 24979403926428.5859

Epoch 3500, Training loss: 26262670104236.4531, Validation loss: 24950644096395.8320

Epoch 4000, Training loss: 26251312309941.2891, Validation loss: 24924117732922.8203

Epoch 4500, Training loss: 26239576923999.7734, Validation lo

tensor([-2.1248e-01, -3.9326e-01,  4.6480e+00,  7.4529e+00,  2.9039e+01,
         4.0698e+01,  1.2172e+02,  1.6630e+02,  3.3453e+02,  3.8658e+02,
         4.8454e+02,  4.8196e+02,  5.0007e+02], requires_grad=True)

In [69]:
#Adam Number 2 learning rate of 00.01
learning_rate_Adam2 = 0.01
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam2 = optim.Adam([params], lr=learning_rate_Adam2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam2,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_ten,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 291183470521275.1875, Validation loss: 53742780790892.7109

Epoch 3, Training loss: 285741537154584.3125, Validation loss: 53127622183921.3906

Epoch 500, Training loss: 26333820848125.9219, Validation loss: 25113197710615.2148

Epoch 1000, Training loss: 26333434841000.7852, Validation loss: 25112483981028.8828

Epoch 1500, Training loss: 26332905483745.6758, Validation loss: 25111466231823.6680

Epoch 2000, Training loss: 26332216627445.3789, Validation loss: 25110105689196.1367

Epoch 2500, Training loss: 26331340378508.3203, Validation loss: 25108337844203.5273

Epoch 3000, Training loss: 26330239881199.4141, Validation loss: 25106076398995.3828

Epoch 3500, Training loss: 26328870133333.1445, Validation loss: 25103213208418.4219

Epoch 4000, Training loss: 26327178631870.0117, Validation loss: 25099617239345.8867

Epoch 4500, Training loss: 26325106538757.4141, Validation lo

tensor([-2.2231e-02,  1.1091e-01,  4.3998e-01,  7.0769e-01,  1.6644e+00,
         2.3079e+00,  5.7309e+00,  8.1246e+00,  2.1031e+01,  2.7578e+01,
         4.6649e+01,  4.6112e+01,  5.0324e+01], requires_grad=True)

In [70]:
#Adam Number 3 learning rate of 0.001
learning_rate_Adam3 = 0.001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam3 = optim.Adam([params], lr=learning_rate_Adam3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam3,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_ten,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 296130908741499.8750, Validation loss: 54302282834905.4609

Epoch 3, Training loss: 295578939806347.1250, Validation loss: 54239849901349.1484

Epoch 500, Training loss: 108986262078127.4531, Validation loss: 33398848066326.6406

Epoch 1000, Training loss: 42963819259218.8281, Validation loss: 26416790971029.3516

Epoch 1500, Training loss: 28159627377137.2266, Validation loss: 25110438523992.1211

Epoch 2000, Training loss: 26422349845908.1055, Validation loss: 25075814375607.5938

Epoch 2500, Training loss: 26335436242132.4609, Validation loss: 25107342692787.2695

Epoch 3000, Training loss: 26333913732294.2109, Validation loss: 25112932847857.4336

Epoch 3500, Training loss: 26333861574808.0977, Validation loss: 25113247655761.3555

Epoch 4000, Training loss: 26333805538794.3945, Validation loss: 25113150922521.2852

Epoch 4500, Training loss: 26333736383223.6445, Validation l

tensor([0.0208, 0.0243, 0.0328, 0.0398, 0.0641, 0.0807, 0.1694, 0.2351, 0.7028,
        1.0857, 3.6526, 3.5206, 5.1243], requires_grad=True)

In [71]:
#Adam Number 4 learning rate of 0.0001
learning_rate_Adam4 = 0.0001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam4 = optim.Adam([params], lr=learning_rate_Adam4)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam4,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_ten,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 296628154150209.5000, Validation loss: 54358528398194.2578

Epoch 3, Training loss: 296572871071625.2500, Validation loss: 54352274980506.0391

Epoch 500, Training loss: 270096059157998.8750, Validation loss: 51360679930647.7656

Epoch 1000, Training loss: 245385502725323.7500, Validation loss: 48575371933863.6328

Epoch 1500, Training loss: 222444863725455.4062, Validation loss: 45996380037033.4375

Epoch 2000, Training loss: 201139615667660.4688, Validation loss: 43608178503270.8750

Epoch 2500, Training loss: 181360422325986.1562, Validation loss: 41398148588049.9219

Epoch 3000, Training loss: 163019511405303.6875, Validation loss: 39356150155466.7344

Epoch 3500, Training loss: 146047148614983.2188, Validation loss: 37474105412357.3672

Epoch 4000, Training loss: 130387675092790.4688, Validation loss: 35745536471723.3672

Epoch 4500, Training loss: 115995969856810.4531, Vali

tensor([0.5421, 0.5421, 0.5422, 0.5422, 0.5422, 0.5423, 0.5424, 0.5426, 0.5436,
        0.5445, 0.5579, 0.5564, 0.5075], requires_grad=True)